# Mixed effect modeling with Scanpy and lme4 R package

In [1]:
import scanpy as sc
import pandas as pd

In [2]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri, r, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

def fit_lme(formula, df, nb=False, **fit_kwargs):
    f = Formula(formula)
    
    lmer = importr('lmerTest') # overloads lmer function from lme4 package
    base = importr('base')
    stats = importr('stats')

    with localconverter(ro.default_converter + pandas2ri.converter):
        if not nb:
            fit = lmer.lmer(f, df, **fit_kwargs)
        else:
            fit = r('lme4::glmer.nb')(f, df, **fit_kwargs)
        anova_df = stats.anova(fit)

    # no automatic pandas conversion here, keep as FloatMatrix
    coef_df = base.summary(fit).rx2('coefficients')

    with localconverter(ro.default_converter + pandas2ri.converter):
        coef_df = r['as.data.frame'](coef_df)

    return coef_df, anova_df

In [3]:
from tqdm.auto import tqdm

def fit_lme_adata(adata, genes, formula, obs_features, use_raw=False):
    coef_df = {}
    anova_df = {}
    covariates = adata.obs[obs_features]
    
    for gene in tqdm(genes):
        gene_vec = adata[:, gene].X if not use_raw else adata.raw[:, gene].X
        df = pd.concat([covariates,
                        pd.DataFrame(gene_vec,
                                     index=adata.obs.index, 
                                     columns=['gene'])], axis=1)

        coefs, anova = fit_lme(formula, df)
        coef_df[gene] = coefs
        anova_df[gene] = anova
        
    coef_df = pd.concat([df.assign(gene=gene) for gene, df in coef_df.items()], axis=0)
    coef_df = coef_df.reset_index().rename(columns={'index': 'fixed_effect'})

    anova_df = pd.concat([df.assign(gene=gene) for gene, df in anova_df.items()], axis=0)
    anova_df = anova_df.reset_index().rename(columns={'index': 'fixed_effect'})
   
    return coef_df, anova_df

In [5]:
adata = sc.datasets.paul15()
adata.obs['somecoef'] = adata[:, 'Cst3'].X
adata.obs.head()

... storing 'paul15_clusters' as categorical
Trying to set attribute `.uns` of view, making a copy.
/Users/gokcen/Library/Python/3.7/lib/python/site-packages/anndata/core/anndata.py:846: FutureWarning: In anndata v0.7+, arrays contained within an AnnData object will maintain their dimensionality. For example, prior to v0.7 `adata[0, 0].X` returned a scalar and `adata[0, :]` returned a 1d array, post v0.7 they will return two dimensional arrays. If you would like to get a one dimensional array from your AnnData object, consider using the `adata.obs_vector`, `adata.var_vector` methods or accessing the array directly.
  warn_flatten()


,paul15_clusters,somecoef
0,7MEP,0.0
1,15Mo,4.0
2,3Ery,2.0
3,15Mo,2.0
4,3Ery,3.0


In [6]:
c, a = fit_lme_adata(adata, ['Gata1', 'Gata2'], 'gene ~ somecoef + (1|paul15_clusters)', ['paul15_clusters', 'somecoef'])

In [7]:
c

,fixed_effect,Estimate,Std. Error,df,t value,Pr(>|t|),gene
0,(Intercept),0.661923,0.230020,18.489866,2.877682,0.009826,Gata1
1,somecoef,0.026600,0.007691,2448.505318,3.458755,0.000552,Gata1
2,(Intercept),0.510597,0.132952,18.049205,3.840475,0.001193,Gata2
3,somecoef,0.001506,0.005124,2221.274718,0.293993,0.768791,Gata2


In [8]:
a

,fixed_effect,Sum Sq,Mean Sq,NumDF,DenDF,F value,Pr(>F),gene
0,somecoef,16.367020,16.367020,1,2448.505318,11.962988,0.000552,Gata1
1,somecoef,0.053109,0.053109,1,2221.274718,0.086432,0.768791,Gata2
